# https://github.com/salesforce/LAVIS/blob/main/lavis/models/blip2_models/blip2_qformer.py#L175

主要变量含义：
- image_embeds_neg：
    - 含义：这是为每个文本选择的负样本图像的embedding。
    - 形状：[bs, D]，其中 bs 是批次大小（batch size），D是图像嵌入的维度。每个元素都是一个负样本图像的嵌入向量。
- text_ids_neg：
    - 含义：这是为每个图像选择的负样本文本的输入ID。
    - 形状：[bs, L]，其中 L 是文本序列的长度。每个元素是一个负样本文本的输入ID序列。
- text_atts_neg：
    - 含义：这是为每个图像选择的负样本文本的mask。所有非pad未知都是1.
    - 形状：[bs, L]，其中 L 是文本序列的长度。每个元素是一个负样本文本的注意力掩码序列。
- text_ids_all：
    - 含义：这是组合后的所有文本输入ID，包括正样本的文本ID（两次）和负样本的文本ID。
    - 形状：[3 * bs, L]，其中 bs 是批次大小（batch size），L 是文本序列的长度。
        - 第一个 bs：正样本的文本ID，来源于 text_tokens.input_ids。
        - 第二个 bs：再次包含正样本的文本ID，仍然来源于 text_tokens.input_ids。
        - 第三个 bs：负样本的文本ID，来源于 text_ids_neg。
- text_atts_all：
    - 含义：这是组合后的所有文本注意力掩码，包括正样本的注意力掩码（两次）和负样本的注意力掩码。
    - 形状：[3 * bs, L]，其中 bs 是批次大小（batch size），L 是文本序列的长度。
        - 第一个 bs：正样本的注意力掩码，来源于 text_tokens.attention_mask。
        - 第二个 bs：再次包含正样本的注意力掩码，仍然来源于 text_tokens.attention_mask。
        - 第三个 bs：负样本的注意力掩码，来源于 text_atts_neg。
- query_tokens_itm:
    - 含义：扩展后的query tokens
    - 形状：和text_ids_all同形状，即[3 * bs, L]
- query_atts_itm:
    - 含义：都是1

In [ ]:
attention_mask_all = torch.cat([query_atts_itm, text_atts_all], dim=1)

queries和文本的mask拼接为一个整体。

In [ ]:
image_embeds_all = torch.cat(
            [image_embeds, image_embeds_neg, image_embeds], dim=0
)  # pos, neg, pos
image_atts_all = torch.ones(image_embeds_all.size()[:-1], dtype=torch.long).to(
    image.device
)

image_embeds_all也是三个batch的拼接。

In [ ]:
output_itm = self.Qformer.bert(
    text_ids_all,
    query_embeds=query_tokens_itm,
    attention_mask=attention_mask_all,
    encoder_hidden_states=image_embeds_all,
    encoder_attention_mask=image_atts_all,
    return_dict=True,
)

# BertModel forward
def forward(
    self,
    input_ids=None,
    attention_mask=None,
    position_ids=None,
    head_mask=None,
    query_embeds=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=None,
    output_hidden_states=None,
    return_dict=None,
    is_decoder=False,
):

In [ ]:
input_ids=text_ids_all
attention_mask=attention_mask_all
query_embeds=query_tokens_itm
encoder_hidden_states=image_embeds_all
encoder_attention_mask=image_atts_all

In [ ]:
# BertModel forward
query_length = query_embeds.shape[1] if query_embeds is not None else 0

embedding_output = self.embeddings(
    input_ids=input_ids,
    position_ids=position_ids,
    query_embeds=query_embeds,
    past_key_values_length=past_key_values_length,
)

input_shape = embedding_output.size()[:-1]

if is_decoder:
    # ...
else:
    extended_attention_mask = self.get_extended_attention_mask(
        attention_mask, input_shape, device, is_decoder
    )

query_length=query数量

embedding_output = torch.cat((query_embeds, input_embeddings), dim=1)

get_extended_attention_mask和图文对比学习的类似。

In [ ]:
# BertModel forward
if encoder_hidden_states is not None:
    if type(encoder_hidden_states) == list:
        # ...
    else:
        (
            encoder_batch_size,
            encoder_sequence_length,
            _,
        ) = encoder_hidden_states.size()
    encoder_hidden_shape = (encoder_batch_size, encoder_sequence_length)

    if type(encoder_attention_mask) == list:
        # ...
    elif encoder_attention_mask is None:
        # ...
    else:
        encoder_extended_attention_mask = self.invert_attention_mask(
            encoder_attention_mask
        )
else:
    # ...

假设 encoder_attention_mask 是一个形如 [0, 0, 1, 1, 1] 的序列，在调用 invert_attention_mask 之后，这个序列可能会被变换成 [1, 1, 0, 0, 0]，或者在一些实现中，可能被变换成一个很大的负数（如 -10000）以确保在 softmax 计算时，这些位置的权重几乎为零。

In [ ]:
# BertModel forward
encoder_outputs = self.encoder(
    embedding_output,
    attention_mask=extended_attention_mask,
    head_mask=head_mask,
    encoder_hidden_states=encoder_hidden_states,
    encoder_attention_mask=encoder_extended_attention_mask,
    past_key_values=past_key_values,
    use_cache=use_cache,
    output_attentions=output_attentions,
    output_hidden_states=output_hidden_states,
    return_dict=return_dict,
    query_length=query_length,
)

# BertEncoder forward
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_values=None,
    use_cache=None,
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True,
    query_length=0,
):

In [ ]:
# BertEncoder forward
hidden_states=embedding_output
attention_mask=extended_attention_mask
encoder_hidden_states=image_embeds_all
encoder_attention_mask=image_atts_all
query_length=query数量

In [ ]:
# BertEncoder forward
for i in range(self.config.num_hidden_layers):
    layer_module = self.layer[i]
    if output_hidden_states:
        all_hidden_states = all_hidden_states + (hidden_states,)

    layer_head_mask = head_mask[i] if head_mask is not None else None
    past_key_value = past_key_values[i] if past_key_values is not None else None

    if getattr(self.config, "gradient_checkpointing", False) and self.training:
        # ...
    else:
        layer_outputs = layer_module(
            hidden_states,
            attention_mask,
            layer_head_mask,
            encoder_hidden_states,
            encoder_attention_mask,
            past_key_value,
            output_attentions,
            query_length,
        )

# BertLayer的forward方法
def forward(
    self,
    hidden_states,
    attention_mask=None,
    head_mask=None,
    encoder_hidden_states=None,
    encoder_attention_mask=None,
    past_key_value=None,
    output_attentions=False,
    query_length=0,
):

In [ ]:
# BertLayer的forward方法
hidden_states=embedding_output
attention_mask=extended_attention_mask
encoder_hidden_states=image_embeds_all
encoder_attention_mask=image_atts_all
query_length=query数量

In [ ]:
# BertLayer的forward方法
self_attention_outputs = self.attention(
    hidden_states,
    attention_mask,
    head_mask,
    output_attentions=output_attentions,
    past_key_value=self_attn_past_key_value,
)